In [ ]:
import os
import gc
from glob import glob
import warnings
import random
import easydict
import copy
from collections import defaultdict
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import itertools
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from tqdm.notebook import tqdm
from PIL import Image
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from torchvision.transforms.functional import to_pil_image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import wandb
import timm
import libtiff

warnings.filterwarnings(action='ignore')
Image.MAX_IMAGE_PIXELS = None
libtiff.libtiff_ctypes.suppress_warnings()
gc.collect()
torch.cuda.empty_cache()
matplotlib.rcParams.update({'font.size': 14})
plt.rcParams['axes.unicode_minus'] = False

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
def seed_everything(random_seed: int):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)

def add_padding(img):
    height, width, _ = img.shape
    img_width, img_height = IMG_SIZE
    fill_value = (0, 0, 0)
    if width / height == img_width / img_height:
        img = cv2.resize(img, (IMG_SIZE))
    elif width / height < img_width / img_height:
        tmp_width = math.ceil(width / (height / img_height))
        img = cv2.resize(img, (tmp_width, img_height))
        to_add = img_width - tmp_width
        left = to_add // 2
        right = to_add - left
        img = cv2.copyMakeBorder(img, 0, 0, left, right, cv2.BORDER_CONSTANT, value=fill_value)
    else:
        tmp_height = math.ceil(height / (width / img_width))
        img = cv2.resize(img, (img_width, tmp_height))
        to_add = img_height - tmp_height
        top = to_add // 2
        bottom = to_add - top
        img = cv2.copyMakeBorder(img, top, bottom, 0, 0, cv2.BORDER_CONSTANT, value=fill_value)
    return img

def get_preprocessing():
    _transform = [
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
    return A.Compose(_transform)

In [ ]:
SEED = 0
IMG_SIZE = (150, 150) 
GPU_IDX = 4
seed_everything(SEED)

model_path = 'pretrained_model_pth'
data_path = 'Data_csv'

In [ ]:
SAVE_PATH_ = 'CAM_savepath'
for folder in ['mask', 'overlay']:
    SAVE_PATH = f'{SAVE_PATH_}/{folder}/'
    if os.path.isdir(SAVE_PATH):
        pass
    else:
        os.mkdir(SAVE_PATH)

In [ ]:
backbone = timm.create_model('resnet50', pretrained=True, num_classes=1024, drop_rate=DROP_RATE)
class ResNet_fc(nn.Module):
    def __init__(self, _backbone, num_classes=2):
        super().__init__()
        
        backbone = copy.deepcopy(_backbone)
        self.backbone = backbone
        self.classifier = nn.Linear(in_features=1024, out_features=num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
df = pd.read_csv(data_path)
img_list = df['img_path'].tolist()
label_list = df['label'].tolist()
prob_list = df['prob'].tolist()

In [ ]:
model = ResNet_fc(backbone, num_classes=2)
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()
model = model.to(GPU_IDX)
target_layers = [model.backbone.layer4[-1]]
c0_target = [ClassifierOutputTarget(0)]
c1_target = [ClassifierOutputTarget(1)]

for idx in tqdm(range(len(img_list))):
    img_path = img_list[idx]
    img_name = img_path.split('/')[-1].split('.')[0]
    label = label_list[idx]
    prob = prob_list[idx]
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)
    xmin, xmax, ymin, ymax = df.loc[df['img_path']==img_path, ['xmin', 'xmax', 'ymin', 'ymax']].values[0]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
    img = img[ymin:ymax, xmin:xmax, :]
    img = add_padding(img)
    input_tensor = get_preprocessing()(image=img)['image'].unsqueeze(0).to(GPU_IDX)
    rgb_img = img / 255.
    if prob >= 0.5:
        c1_grayscale_cam = cam(input_tensor=input_tensor, targets=c1_target)
        c1_grayscale_cam = c1_grayscale_cam[0, :]
        c1_visualization = show_cam_on_image(rgb_img, c1_grayscale_cam, use_rgb=True)
        c1_grayscale_cam = (c1_grayscale_cam * 255).astype(np.uint8)
    else:
        c0_grayscale_cam = cam(input_tensor=input_tensor, targets=c0_target)
        c0_grayscale_cam = c0_grayscale_cam[0, :]
        c0_visualization = show_cam_on_image(rgb_img, c0_grayscale_cam, use_rgb=True)
        c0_grayscale_cam = (c0_grayscale_cam * 255).astype(np.uint8)